In [1]:
from bb2022_functions import *
%matplotlib inline
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
md = pd.read_csv("metadata_merged.csv")
merged = pd.read_csv("metadata_niskin.csv")
all_md = pd.read_csv("allmetadata.csv")

In [ ]:
detect_anomalies('Temperature_y', all_md, 1, yr={2022}, month={1,2,3,4,5})

In [6]:
d1 = all_md.loc[all_md['depth'] == 1]
d1 = d1[['weekn', 'depth', 'Phosphate', 'Silicate', 'Nitrate',
       'Ammonia', 'Chlorophyll A']]
d1.rename(columns={'Temperature_x': 'Temperature'},
          inplace= True)

In [7]:
d1 = d1.melt(id_vars=['weekn', 'depth'])

In [ ]:
def get_stage(weekNb):
    if weekNb <8:
        return 'Pre-bloom'
    elif weekNb >= 8:
        return 'Bloom'

d1['Time'] = d1['weekn'].apply(get_stage)

In [ ]:
df1 = d1[d1['Time'] == 'Bloom']
df2 = d1[d1['Time'] == 'Pre-bloom']

In [ ]:
sns.lmplot(
    data = d1,
    x="weekn", y="value",
    hue = 'variable', palette = 'viridis', scatter=True
)

In [ ]:
sns.lmplot(x='weekn', y='value', data=df1, hue='variable') 
sns.lmplot(x='weekn', y='value', data=df2, hue='variable')

In [ ]:
d1.drop_duplicates(inplace=True)
d1.set_index('weekn', inplace=True)

In [ ]:
rolling_mean = d1.rolling(5).mean()
rolling_std = d1.rolling(5).std()

In [ ]:
plt.plot(d1, color="blue",label="Temperature")
plt.plot(rolling_mean, color="red", label="Rolling Mean Temperature")
plt.plot(rolling_std, color="black", label = "Rolling Standard Deviation Temperate")
plt.title("Passenger Time Series, Rolling Mean, Standard Deviation")
#plt.legend(loc="upper right")

In [ ]:
autocorrelation_lag1 = d1['Temperature'].autocorr(lag=1)
print("One Month Lag: ", autocorrelation_lag1)

autocorrelation_lag3 = d1['Temperature'].autocorr(lag=3)
print("Three Month Lag: ", autocorrelation_lag3)

autocorrelation_lag6 = d1['Temperature'].autocorr(lag=6)
print("Six Month Lag: ", autocorrelation_lag6)

autocorrelation_lag9 = d1['Temperature'].autocorr(lag=9)
print("Nine Month Lag: ", autocorrelation_lag9)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decompose = seasonal_decompose(d1['Temperature'],model='multiplicative', period=3)
trend = decompose.trend
residual = decompose.resid

decompose.plot();
plt.show()

In [ ]:
sns.relplot(
    data=d1, x="weekn", y="value",
    row="depth", hue="variable",
    kind="line"
)